# Building a Simple Chatbot using LangChain and OpenAI

The purpose of this notebook is to show you how to build a simple chatbot using [LangChain](https://www.langchain.com/), and [OpenAI](https://openai.com/). The code is based on the ["Build a Chatbot"](https://python.langchain.com/v0.2/docs/tutorials/chatbot/) LangChain tutorial, but with modifications to store the chat history on a SQLite database.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from os import environ

if not environ.get("OPENAI_API_KEY"):
    from getpass import getpass
    environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY")

In [3]:
from functools import partial
from pathlib import Path
from random import choices
from string import ascii_lowercase

import gradio as gr

from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

In [4]:
# Initialize the OpenAI chat model (replace "gpt-4o" with your desired model)
model = ChatOpenAI(model="gpt-4o")

In [5]:
# Define the chat prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a virtual assistant named 'SimpleChatBot'. You are an expert on video-games focused on point-and-click adventure games."
        ),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template(template="{input}")
    ]
)

In [6]:
# Build the chain of processing steps for the chatbot
chain = (
    RunnablePassthrough.assign(
        messages=lambda x: x["history"][-10:]
    )
    | prompt
    | model
)

In [7]:
connection_string = "sqlite:///simple_chatbot.db"

# Delete the database file if it exists (optional, for clean start)
Path("simple_chatbot.db").unlink(missing_ok=True)

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """
    This function retrieves the chat message history for a specific session ID.
    It uses the SQLChatMessageHistory class from the langchain_community library.
    """
    return SQLChatMessageHistory(session_id, connection_string)

# Wrap the chain with message history handling
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [8]:
def chat_interface_fn(
    runnable: Runnable,
    message: str,
    history: list[list[str]],  # We don't use the ChatInterface history
    session_id: str
):
    """
    ChatInterface fn argument. This is invoked everytime the
    "Submit" button is pressed on the Gradio interface.
    """
    response = runnable.invoke(
        {"input": message},
        config={"configurable": {"session_id": session_id}},
    )
    return response.content


gr.ChatInterface(
    fn=partial(chat_interface_fn, with_message_history),
    additional_inputs=[
        gr.Textbox("".join(choices(ascii_lowercase, k=8)), label="Session ID"),
    ],
).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
